In [1]:
%%writefile app.py
import streamlit as st
import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

# Configure Wikipedia settings
wikipedia.set_lang("en")
wikipedia.set_rate_limiting(rate_limit=True)

# Cache Wikipedia data to improve performance
@st.cache_data(show_spinner=False, ttl=3600)
def get_wikipedia_data(query, sentences=3):
    try:
        page = wikipedia.page(query, auto_suggest=False)
        return {
            "title": page.title,
            "summary": wikipedia.summary(query, sentences=sentences),
            "url": page.url,
            "sections": page.sections,
            "images": page.images[0] if page.images else None,
            "references": page.references
        }
    except (DisambiguationError, PageError) as e:
        return {"error": str(e), "options": e.options if isinstance(e, DisambiguationError) else []}

# App layout and configuration
st.set_page_config(page_title="Wiki Assistant", page_icon="📚")
st.title("📚 Wikipedia Q&A Expert")
st.markdown("### Get precise answers with verified Wikipedia sources")

# Sidebar controls
with st.sidebar:
    st.header("Settings")
    max_sentences = st.slider("Answer length (sentences)", 1, 10, 3)
    show_advanced = st.checkbox("Show advanced options")

# Main interface
query = st.text_input("Ask about anything:", 
                     placeholder="e.g., 'Roman Empire formation'",
                     help="Be specific for better results!")

if st.button("Research") or query:
    if not query.strip():
        st.warning("Please enter a question")
        st.stop()

    with st.spinner("🔍 Searching Wikipedia..."):
        try:
            # First try direct search
            results = wikipedia.search(query, results=5)
            if not results:
                st.error("No Wikipedia pages found. Try different keywords.")
                st.stop()

            # Handle disambiguation
            if len(results) > 1 and show_advanced:
                selected_page = st.selectbox(
                    "Multiple matches found. Select the correct page:",
                    results
                )
            else:
                selected_page = results[0]

            data = get_wikipedia_data(selected_page, sentences=max_sentences)

            if "error" in data:
                if "options" in data:
                    st.error(f"Ambiguous term. Try being more specific. Options: {', '.join(data['options'][:5])}")
                else:
                    st.error("Page not found. Please refine your query.")
                st.stop()

            # Display results
            st.success(f"## {data['title']}")
            
            # Main content columns
            col1, col2 = st.columns([3, 1])
            with col1:
                st.markdown(f"**Answer:**\n\n{data['summary']}")
                
                if show_advanced and data['sections']:
                    st.markdown("\n**Page Sections:**")
                    st.write(", ".join(data['sections'][:5]))
                
                st.markdown(f"\n**Source:** [📖 Read full article]({data['url']})")

            with col2:
                if data['images']:
                    st.image(data['images'], caption=data['title'], use_column_width=True)

            # Advanced features
            if show_advanced:
                with st.expander("Technical Details"):
                    st.write(f"**Page URL:** {data['url']}")
                    if data['references']:
                        st.markdown("**Key References:**")
                        for ref in data['references'][:3]:
                            st.write(f"- {ref}")

        except Exception as e:
            st.error(f"Research failed: {str(e)}")
            st.info("💡 Tip: Try simplifying your question or check spelling")

st.markdown("---")
st.caption("Note: This tool uses Wikipedia content under CC BY-SA 3.0 license. Accuracy depends on Wikipedia's content quality.")

# Run with: streamlit run app.py

Overwriting app.py


In [ ]:
!streamlit run app.py